In [1]:
!pip install bitsandbytes

  Using cached torch-2.4.1-cp312-cp312-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached numpy-2.1.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.13.2-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached setuptools-74.1.2-py3-none-any.whl.metadata (6.7 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x8

In [2]:
!pip install -U bitsandbytes

In [3]:
!pip install transformers

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.24.6-py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2024.7.24-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached safetensors-0.4.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tokenizers-0.19.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.8-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2

In [6]:
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 970.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [7]:
!pip install ipywidgets

  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.13-py3-none-any.whl (2.3 MB)

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Исключительно ознакомительный пример.
# НЕ НАДО ТАК ИНФЕРИТЬ МОДЕЛЬ В ПРОДЕ.
# Когда есть https://github.com/vllm-project/vllm

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
#     load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    # device_map="auto"
    low_cpu_mem_usage=True
)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model-00002-of-00004.safetensors:  56%|#####5    | 2.80G/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
instruction = "Каковы основные причины начала Второй мировой войны?"
messages = [
    {"role": "user", "content": f"{instruction}"}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)
print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

In [5]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

inputs = [
    "Сократи следующий текст: В рамках городского проекта «Профессиональное обучение без границ» востребованные " + 
    "профессии (вожатый, оператор ЭВМ, цифровой куратор, фотограф) получили 337 учащихся 9–11-х классов. В целях " +
    "подготовки учащихся к осознанному выбору профессии школа активно сотрудничает с ведущими московскими вузами. " + 
    "В их число вошли Российский национальный исследовательский медицинский университет имени Н.И. Пирогова, Первый " + 
    "московский государственный медицинский университет имени И.М. Сеченова, Российский университет дружбы народов "+
    "имени Патриса Лумумбы, Национальный исследовательский университет «Высшая школа экономики», Московский "+
    "государственный университет имени М.В. Ломоносова, МИРЭА — Российский технологический университет, Университет "+
    "науки и технологий МИСИС, Российский экономический университет имени Г.В. Плеханова, Финансовый университет при "+
    "Правительстве Российской Федерации. В учреждении обучаются свыше 14,1 тысячи детей, в том числе 9,8 тысячи школьников " +
    "и более 4,3 тысячи дошкольников. Трудовой коллектив насчитывает 1202 сотрудника, включая 439 учителей и 282 воспитателя."]
for query in inputs:
    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": DEFAULT_SYSTEM_PROMPT
    }, {
        "role": "user",
        "content": query
    }], tokenize=False, add_generation_prompt=True)
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    print(query)
    print(output)
    print()
    print("==============================")
    print()

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}

Сократи следующий текст: В рамках городского проекта «Профессиональное обучение без границ» востребованные профессии (вожатый, оператор ЭВМ, цифровой куратор, фотограф) получили 337 учащихся 9–11-х классов. В целях подготовки учащихся к осознанному выбору профессии школа активно сотрудничает с ведущими московскими вузами. В их число вошли Российский национальный исследовательский медицинский университет имени Н.И. Пирогова, Первый московский государственный медицинский университет имени И.М. Сеченова, Российский университет дружбы народов имени Патриса Лумумбы, Национальный исследовательский университет «Высшая школа экономики», Московский государственный университет имени М.В. Ломоносова, МИРЭА — Российский технологический университет, Университет науки и техн

In [6]:
output

'Городской проект "Профессиональное обучение без границ" охватил 337 учащихся 9-11 классов по востребованным специальностям (вожатый, оператор ЭВМ, цифровой куратор, фотограф). Школа сотрудничает с ведущими московскими вузами, такими как РНИМУ им. Н.И. Пирогова, РУДН им. Патриса Лумумбы и другие. Обучается более 14 тысяч детей, из них 9,8 тысячи школьников и 4,3 тысячи дошкольников. В трудовом коллективе 1202 сотрудника, включая 439 учителей и 282 воспитателя.'